In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0, "/home/ninarell/OneDrive/WF_GAN_FOR_GLASSES/B_GEN/bgflow")
import bgflow
sys.path.insert(0, "/home/ninarell/OneDrive/WF_GAN_FOR_GLASSES/B_GEN/anode")
import anode

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from bgflow.utils import (assert_numpy, length_ppp, 
                          remove_mean, IndexBatchIterator, LossReporter, as_numpy, compute_distances, distance_vectors, distances_from_vectors, length_ppp
)
from bgflow import (GaussianMCMCSampler, DiffEqFlow, BoltzmannGenerator, Energy, Sampler, 
                    MultiDoubleWellPotential, MeanFreeNormalDistribution, KernelDynamics)

from glob import glob

In [ ]:
#fnames = glob('/home/ninarell/OneDrive/WF_GAN_FOR_GLASSES/LJ_CRYSTAL/T_0.700_box/dumplin/dump.npt_nose_T1.0_P0.*.lammpstrj')
#fnames = fnames + glob('/home/ninarell/OneDrive/WF_GAN_FOR_GLASSES/LJ_CRYSTAL/T_0.700_box/dumplin/dump.npt_nose_T0.700_P0.0000020000.lammpstrj')
#fnames = glob('/home/ninarell/OneDrive/WF_GAN_FOR_GLASSES/LJ_CRYSTAL/T_0.700_box/dumplin/dump.npt_nose_T1.0_P0.0000000000.lammpstrj')
#arrays = [np.loadtxt(f, skiprows=9)[:,2:5] for f in fnames]
#coordinates = np.array(arrays)
temperature = float(1.00)
side= 4 #3.66
n_particles = 4 #len(coordinates[0])
spacial_dim = 2
dim_ics = n_particles * spacial_dim
#coordinates=coordinates.reshape(len(arrays), dim_ics)

In [ ]:
from  bgflow.distribution.energy import LennardJonesPotentialPPP
from  bgflow.distribution.energy import LennardJonesPotential
rm = 2**(1./6.)
target = LennardJonesPotentialPPP(dim = dim_ics, n_particles = n_particles, side = side, oscillator = False, rm=rm, two_event_dims=False)
#target = LennardJonesPotential(dim = dim_ics, n_particles = n_particles,oscillator = False,  two_event_dims=False)

In [ ]:
def plot_energy(coordinates, target):
    xs = torch.Tensor(coordinates)
    #xs = xs.view(-1,10,3)
    energy = target.energy(xs).detach().numpy()
    x=np.arange(1,len(energy)+1)

    fig = plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(x, energy)

    plt.subplot(1, 2, 2)
    counts, bins = np.histogram(energy, density=True)
    plt.yscale("log")
    plt.stairs(counts, bins)

In [ ]:
def plot_energy(coordinates, target):
    xs = torch.Tensor(coordinates)
    #xs = xs.view(-1,10,3)
    energy = target.energy(xs).detach().numpy()
    x=np.arange(1,len(energy)+1)

    fig = plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(x, energy)

    plt.subplot(1, 2, 2)
    counts, bins = np.histogram(energy, density=True)
    plt.yscale("log")
    plt.stairs(counts, bins)

In [ ]:
# #Box constraint
def constraint(x):
    return length_ppp(x, side)

In [ ]:
import random
import numpy as np

def generate_random_points(num_points, x_range, y_range):
    points = np.empty((num_points, 2))
    for i in range(num_points):
        x = random.uniform(x_range[0], x_range[1])
        y = random.uniform(y_range[0], y_range[1])
        points[i] = [x, y]
    return points

# Parameters
num_points = n_particles  # Number of points to generate
x_range = (-side, side)  # Range for x coordinates
y_range = (-side, side)  # Range for y coordinates

# Generate points
random_points = generate_random_points(num_points, x_range, y_range)

# Print the generated points
random_points=random_points.reshape(dim_ics)

In [ ]:
# define a MCMC sampler to sample from the target energy

init_state = torch.Tensor(random_points) #zeros(10,3)

mcsampler = GaussianMCMCSampler(target, init_state=init_state, temperature=temperature, box_constraint=constraint, noise_std=0.15)
#mcsampler = GaussianMCMCSampler(target, init_state=init_state, temperature=temperature, noise_std=0.15)
mcsampler.sample(1000)
data = mcsampler.sample(4096)
#data = remove_mean(data, n_particles, spacial_dim)
while torch.any(abs(data) > side):
    data = length_ppp(data, side)
    data = remove_mean(data, n_particles, spacial_dim)
data = data.view(-1,dim_ics)

In [ ]:
plot_energy(data, target)

In [ ]:
# now set up a prior

from bgflow import NormalDistribution, TruncatedNormalDistribution, MeanFreeNormalDistribution, CircularNormalDistribution

#mean = torch.Tensor(data[10]).view(dim_ics)
#mean = torch.ones(dim_ics)*0

#cov = torch.zeros(dim_ics, dim_ics)
#cov = cov.fill_diagonal_(0.004)

#prior = NormalDistribution(dim_ics, mean=mean, cov=cov, side=side)
prior =  MeanFreeNormalDistribution(dim_ics, n_particles, 0.005, two_event_dims=False, side=side)

#mean=torch.ones(dim_ics)*0.01
#sigma=torch.ones(dim_ics)*0.2
#prior = CircularNormalDistribution(mu=mean, sigma=sigma)

# upper_bound=torch.ones(dim_ics)*side
# lower_bound=-upper_bound
# sigma=torch.ones(dim_ics)*0.02
# prior = TruncatedNormalDistribution(mu=mean, sigma=sigma, lower_bound=lower_bound, upper_bound=upper_bound,  sampling_method = "rejection" )
#prior = TruncatedNormalDistribution(mu=mean, sigma=sigma, lower_bound=lower_bound, upper_bound=upper_bound,  is_learnable=True )

In [ ]:
#data_prior = prior._sample_with_temperature(10000, temperature=temperature)
data_prior = prior.sample(1000)
#data_prior = data_prior.view(-1, 10, 3)
plot_energy(data_prior, target)

In [ ]:
plt.xlim(-side*2,side*2)
plt.ylim(-side*2,side*2)
plt.scatter(data_prior[:,0], data_prior[:,1])
plt.scatter(data_prior[:,2], data_prior[:,3])
plt.scatter(data_prior[:,4], data_prior[:,5])
plt.scatter(data_prior[:,6], data_prior[:,7])
plt.plot([-side,-side], [-side,side], linestyle="--", color='black')
plt.plot([side,side], [-side,side], linestyle="--", color='black')
plt.plot([-side,side], [-side,-side], linestyle="--", color='black')
plt.plot([-side,side], [side,side], linestyle="--", color='black')

In [ ]:
# set of the equivariant kernel dynamics

n_dimensions = spacial_dim
d_max = 8
mus = torch.linspace(0, d_max, 50) #.cuda()

mus.sort()
gammas = 0.3 * torch.ones(len(mus)) #.cuda()

mus_time = torch.linspace(0, 1, 10) #.cuda()
gammas_time = 0.3 * torch.ones(len(mus_time)) #.cuda()


kdyn = KernelDynamics(n_particles, n_dimensions, mus, gammas, optimize_d_gammas=True, optimize_t_gammas=True,
                      mus_time=mus_time, gammas_time=gammas_time, periodic = True, side = side) #.cuda()


In [ ]:
flow = DiffEqFlow(dynamics = kdyn, side=side)

In [ ]:
# having a flow and a prior, we can now define a Boltzmann Generator

bg = BoltzmannGenerator(prior, flow, target) #.cuda()

In [ ]:
# use DTO in the training process
flow._use_checkpoints = True

# Anode options
options={
    "Nt": 3,
    "method": "RK4"
}
flow._kwargs = options

In [ ]:
# initial training with likelihood maximization on data set

n_kl_samples = 64
n_batch = 64
batch_iter = IndexBatchIterator(len(data), n_batch)

optim = torch.optim.Adam(bg.parameters(), lr=1e-4)

n_epochs = 10
n_report_steps = 4

# mixing parameter
lambdas = torch.linspace(1., 0.1, n_epochs) #.cuda()

reporter = LossReporter("NLL", "KLL")

In [ ]:
for epoch, lamb in enumerate(lambdas):
    for it, idxs in enumerate(batch_iter):
        batch = data[idxs] #.cuda()
        
        optim.zero_grad()

        # negative log-likelihood of the batch is equal to the energy of the BG
        nll = bg.energy(batch).mean()
        # aggregate weighted gradient
        (lamb * nll).backward()
        
        # kl divergence to the target
        kll = bg.kldiv(n_kl_samples).mean()

        # aggregate weighted gradient
        ((1. - lamb) * kll).backward()
        
        reporter.report(nll, kll)
        
        optim.step()
        
        
        if it % n_report_steps == 0:
            print("\repoch: {0}, iter: {1}/{2}, lambda: {3}, NLL: {4:.4}, KLL: {5:.4}".format(
                    epoch,
                    it,
                    len(batch_iter),
                    lamb,
                    *reporter.recent(1).ravel()
                ), end="")

In [ ]:
reporter.plot()

In [ ]:
n_samples = 2000
samples, latent, dlogp = bg.sample(n_samples, with_latent=True, with_dlogp=True)
log_w = as_numpy(bg.log_weights_given_latent(samples, latent, dlogp))

In [ ]:
plot_energy(samples, target)

In [ ]:
#samples = remove_mean(samples, n_particles, spacial_dim)
r = distance_vectors(samples.view(-1, n_particles, spacial_dim))
distances_x = as_numpy(distances_from_vectors(length_ppp(r, side)))

In [ ]:
print(log_w.shape)
print(dlogp.shape)
print(latent.shape)
print(samples.shape)
print(distances_x.reshape(-1).shape)

In [ ]:
plt.figure(figsize=(16,9))
#plt.plot(d, e, label="Groundtruth", linewidth=4, alpha = 0.5)
#data = remove_mean(data, n_particles, spacial_dim)
r = distance_vectors(data.view(-1, n_particles, spacial_dim))
dists_data = as_numpy(distances_from_vectors(length_ppp(r,  side)))
#data_prior = remove_mean(data_prior, n_particles, spacial_dim)
r = distance_vectors(data_prior.view(-1, n_particles, spacial_dim))
dists_data_prior = as_numpy(distances_from_vectors(length_ppp(r,  side)))


plt.hist(dists_data.reshape(-1), bins=50, label="training samples", alpha=0.5, density=True, histtype='step', linewidth=4);
plt.hist(dists_data_prior.reshape(-1), bins=50, label="prior samples", alpha=0.5, density=True, histtype='step', linewidth=4);
plt.hist(distances_x.reshape(-1), bins=50, label="bg samples", alpha=0.7, density=True, histtype='step', linewidth=4);
#plt.hist(distances_x.reshape(-1), bins=50, label="reweighted bg samples", alpha=0.7, density=True, histtype='step', linewidth=4, weights=np.exp(log_w));
plt.xlim(0,np.sqrt(spacial_dim*side*side)+2)
plt.legend(fontsize=35)
plt.xlabel("u(x)", fontsize=45)  
plt.xticks(fontsize=45) 
plt.yticks(fontsize=45);

In [ ]:
np.where(np.any(distances_x> np.sqrt(side*side*2)))

In [ ]:
plt.xlim(-side*3,side*3)
plt.ylim(-side*3,side*3)
samplesnp = samples.detach().cpu().numpy()
plt.scatter(samplesnp[:,0], samplesnp[:,1])
plt.scatter(samplesnp[:,2], samplesnp[:,3])
plt.scatter(samplesnp[:,4], samplesnp[:,5])
plt.scatter(samplesnp[:,6], samplesnp[:,7])
plt.plot([-side,-side], [-side,side], linestyle="--", color='black')
plt.plot([side,side], [-side,side], linestyle="--", color='black')
plt.plot([-side,side], [-side,-side], linestyle="--", color='black')
plt.plot([-side,side], [side,side], linestyle="--", color='black')

In [ ]:
plt.xlim(-side*2,side*2)
plt.ylim(-side*2,side*2)
datanp = data.detach().cpu().numpy()
plt.scatter(datanp[:,0], datanp[:,1])
plt.scatter(datanp[:,2], datanp[:,3])
plt.scatter(datanp[:,4], datanp[:,5])
plt.scatter(datanp[:,6], datanp[:,7])
plt.plot([-side,-side], [-side,side], linestyle="--", color='black')
plt.plot([side,side], [-side,side], linestyle="--", color='black')
plt.plot([-side,side], [-side,-side], linestyle="--", color='black')
plt.plot([-side,side], [side,side], linestyle="--", color='black')

In [ ]:
r= torch.linspace(0.5, 5, 100)
def lennard_jones_energy_torch(r, eps=1.0, rm=1.0, rct=2.5):

    r = torch.where((r < rct).clone().detach(), r, torch.tensor(1000000.))
    r_int=0.8
    b= 0.02
    r = torch.where((r < r_int).clone().detach(), b * r / r_int + r_int - b  , r)
    return eps * ((rm / r) ** 12 - 2 * (rm / r) ** 6) / 2.

In [ ]:
lj = lennard_jones_energy_torch(r)
plt.plot(r, lennard_jones_energy_torch(r))

In [ ]:
plt.figure(figsize=(16,9))

distances_x = as_numpy(compute_distances(samples,  n_particles, n_dimensions))
dist_data = as_numpy(compute_distances(data,  n_particles, n_dimensions))
dist_data_prior = as_numpy(compute_distances(data_prior,  n_particles, n_dimensions))

plt.hist(dists_data.reshape(-1), bins=50, label="training samples", alpha=0.5, density=True, histtype='step', linewidth=4);
plt.hist(dists_data_prior.reshape(-1), bins=50, label="prior samples", alpha=0.5, density=True, histtype='step', linewidth=4);
plt.hist(distances_x.reshape(-1), bins=50, label="bg samples", alpha=0.7, density=True, histtype='step', linewidth=4);
#plt.hist(distances_x.reshape(-1), bins=50, label="reweighted bg samples", alpha=0.7, density=True, histtype='step', linewidth=4, weights=np.exp(log_w));
plt.xlim(0,np.sqrt(spacial_dim*side*side)+5)
plt.legend(fontsize=35)
plt.xlabel("u(x)", fontsize=45)  
plt.xticks(fontsize=45) 
plt.yticks(fontsize=45);

In [ ]:
print(log_w.shape)
print(dlogp.shape)
print(latent.shape)
print(samples.shape)
print(distances_x.shape)